In [30]:
import msprime
from trails.cutpoints import cutpoints_ABC, cutpoints_AB
import re
from ray import shutdown
# %load_ext rpy2.ipython

In [31]:
import pandas as pd
import numpy as np

n_int_AB = 1
n_int_ABC = 1

mu = 1.25e-8
g = 29
N_AB = 7_000*2*mu
N_BC = 7_000*2*mu
N_ABC = 7_000*2*mu
N_ref = N_ABC
t_1 = 62_041/g*mu
# t_2 = (580_000-t_1)/g*mu
# t_3 = (6_000_000-t_2)/g*mu
t_2 = 580_000/g*mu-t_1
t_3 = 6_000_000/g*mu-t_2
t_A = t_1
t_B = t_1
t_C = t_1+t_2
t_m = t_1-54_000/g*mu
t_upper = t_3-cutpoints_ABC(n_int_ABC,  1/N_ABC)[-2]
t_out = t_1+t_2+t_3+2*N_ABC
r = 1e-8/mu
mu = mu/mu
m = 0.02

seed = 2

In [32]:
m*(2/3)*np.exp(-(t_2+t_m)/(N_AB))+(1-m)*(2/3)*np.exp(-(t_2)/(N_BC))

0.186072577384884

In [35]:
from trails.optimizer import trans_emiss_calc_introgression

transitions, emissions, starting, hidden_states, observed_states = trans_emiss_calc_introgression(
    t_1, t_1, t_C, t_2, t_upper, t_out, t_m,
    N_AB, N_BC, N_ABC, r, m, n_int_AB, n_int_ABC)

shutdown()

transitions

array([[9.99948446e-01, 1.71003529e-05, 1.71003529e-05, 1.71003529e-05,
        2.53250729e-07],
       [1.29832218e-04, 9.99703523e-01, 8.20284476e-05, 8.19190624e-05,
        2.69697622e-06],
       [1.29832218e-04, 8.20284476e-05, 9.99696952e-01, 8.84908558e-05,
        2.69697622e-06],
       [1.29832218e-04, 8.19190624e-05, 8.84908558e-05, 9.99697061e-01,
        2.69697622e-06],
       [1.23157256e-05, 1.72746404e-05, 1.72746404e-05, 1.72746404e-05,
        9.99935860e-01]])

In [63]:
m*(2/3)*np.exp(-(t_2+t_m)/(N_AB))+(1-m)*(2/3)*np.exp(-(t_2)/(N_BC))

0.186072577384884

In [62]:
acc = 0
for i in hidden_states:
    if hidden_states[i][0] in [2, 3]:
        acc += starting[i]
acc

0.18607257738488395

In [39]:
pd.DataFrame(transitions, columns=[str(i) for i in hidden_states.values()]).to_csv(f'tables/transitions_{n_int_ABC}.csv', index = False)

In [40]:
n_sites = 1_000_000

demography = msprime.Demography()
demography.add_population(name="A", initial_size=N_AB, default_sampling_time=t_1-t_A)
demography.add_population(name="B", initial_size=N_AB, default_sampling_time=t_1-t_B)
demography.add_population(name="B_anc", initial_size=N_AB, initially_active=False)
demography.add_population(name="C", initial_size=N_BC, default_sampling_time=t_1+t_2-t_C)
demography.add_population(name="D", initial_size=N_AB, default_sampling_time=t_1-t_1)
demography.add_population(name="AB", initial_size=N_AB)
demography.add_population(name="ABC", initial_size=N_ABC)
demography.add_population(name="ABCD", initial_size=N_ABC)
demography.add_admixture(time = t_1-t_m, derived="B", ancestral=["B_anc", "C"], proportions=(1-m, m))
demography.add_population_split(time=t_1, derived=["A", "B_anc"], ancestral="AB")
demography.add_population_split(time=t_1+t_2, derived=["AB", "C"], ancestral="ABC")
demography.add_population_split(time=t_1+t_2+t_3, derived=["ABC", "D"], ancestral="ABCD")

# demography.debug()

ts = msprime.sim_ancestry(
    {"A": 1, "B": 1, "C": 1,
     "D": 1
    },
    demography=demography,
    recombination_rate=r,
    sequence_length=n_sites,
    ploidy=1,
    random_seed=seed
)



In [41]:
# Add mutations
dct = {v: k for k, v in observed_states.items()}

mutated_ts = msprime.sim_mutations(ts, rate=mu, random_seed=seed)

nochange_lst = [dct['AAAA'], dct['CCCC'], dct['TTTT'], dct['GGGG']]
sim_genome = np.random.choice(nochange_lst, n_sites)

mut_lst = []
mut_loc = []
for variant in mutated_ts.variants():
    mut_loc.append(variant.site.position)
    mut_lst.append(''.join([variant.alleles[i] for i in variant.genotypes]))

for i in range(len(mut_loc)):
    sim_genome[int(mut_loc[i])] = dct[mut_lst[i]]

E = sim_genome

In [42]:
# from trails.optimizer import loglik_wrapper

# loglik = loglik_wrapper(transitions, emissions, starting, [E])

# loglik

In [43]:
from trails.optimizer import post_prob_wrapper

postprobs = post_prob_wrapper(transitions, emissions, starting, [E])


In [44]:
postprobs_df = pd.DataFrame(postprobs[0], columns=[str(i) for i in hidden_states.values()])
postprobs_df = postprobs_df[0:1000000]

In [45]:
postprobs_df

,"(0, 0, 0)","(1, 0, 0)","(2, 0, 0)","(3, 0, 0)","(4, 0, 0)"
0,0.988950,0.004027,0.003505,0.003504,0.000014
1,0.988956,0.004025,0.003503,0.003502,0.000014
2,0.988961,0.004024,0.003501,0.003501,0.000014
3,0.988967,0.004022,0.003499,0.003499,0.000014
4,0.988972,0.004020,0.003497,0.003497,0.000014
...,...,...,...,...,...
999995,0.282007,0.157284,0.180636,0.344756,0.035316
999996,0.281992,0.157306,0.180653,0.344730,0.035319
999997,0.281976,0.157328,0.180670,0.344704,0.035322
999998,0.281960,0.157350,0.180687,0.344678,0.035324


In [46]:
# import demesdraw
# import matplotlib.pyplot as plt

In [47]:
# graph = msprime.Demography.to_demes(demography)
# fig, ax = plt.subplots()  # use plt.rcParams["figure.figsize"]
# demesdraw.tubes(graph, ax=ax, seed=1)
# plt.show()

In [48]:
# ts.draw_svg(y_axis=True)

In [49]:
cut_ABC = t_1+t_2+cutpoints_ABC(n_int_ABC, 1/N_ABC)
cut_AB = t_1+cutpoints_AB(n_int_AB, t_2, 1/N_AB)
cut_AB_2 = np.array([t_1-t_m]+list(cut_AB)[1::])

left_lst = []
right_lst = []
tree_state = []
t_AB_vec = []
t_ABC_vec = []
for t in ts.trees():
    # Append start coordinate
    left_lst.append(t.interval.left)
    # Append end coordinate
    right_lst.append(t.interval.right-1)
    # Get all non-zero coalescent times
    ntimes = [ts.nodes()[n].time for n in t.nodes() if ts.nodes()[n].time not in [0, t_1-t_A, t_1-t_B, t_1-t_C]]
    ntimes = sorted(ntimes)
    # Get time of the first event
    mint = ntimes[0]
    mint2 = ntimes[1]
    # Find topology
    find_re = re.findall("n\d,n\d", t.as_newick(include_branch_lengths=False))[0]
    # Sort species within topology
    find_re = sorted(find_re.split(','))
    # If V0 or V1
    if find_re == ['n0', 'n1']:
        # If the time of the first coalescent is larger than the deepest speciation event
        if mint>=(t_1+t_2):
            state = (1, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
            # Append V1 state
        else:
            state = (0, (mint>cut_AB).sum()-1, (mint2>cut_ABC).sum()-1)
            # Append V0 state
    # If V2
    elif find_re == ['n0', 'n2']:
        state = (2, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
    # If V3
    elif find_re == ['n1', 'n2']:
        if mint>=(t_1+t_2):
            state = (3, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
        else:
            state = (4, (mint>cut_AB_2).sum()-1, (mint2>cut_ABC).sum()-1)
    else:
        state = (5, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1)
        print((find_re, (mint>cut_ABC).sum()-1, (mint2>cut_ABC).sum()-1))
    tree_state.append(state)
    t_AB_vec.append(mint)
    t_ABC_vec.append(mint2)

In [50]:
set(tree_state)

{(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0)}

In [51]:
df = pd.DataFrame()
df['state'] = [str(i) for i in tree_state]
df['start'] = left_lst
df['end'] = right_lst
df

,state,start,end
0,"(0, 0, 0)",0.0,1497.0
1,"(0, 0, 0)",1498.0,3105.0
2,"(0, 0, 0)",3106.0,8300.0
3,"(0, 0, 0)",8301.0,10665.0
4,"(0, 0, 0)",10666.0,14327.0
...,...,...,...
892,"(3, 0, 0)",987168.0,991862.0
893,"(0, 0, 0)",991863.0,993045.0
894,"(0, 0, 0)",993046.0,998161.0
895,"(0, 0, 0)",998162.0,999065.0


In [52]:
df.to_csv(f'./tables/df_{n_int_ABC}.csv', index = False)
postprobs_df.to_csv(f'./tables/postprobs_df_{n_int_ABC}.csv', index = False)